In [1]:
using Pkg
Pkg.activate("climate"; shared=true)

"/Users/imolk/Library/Julia/environments/climate/Project.toml"

In [2]:
ICAO = "KWRB"
USAF, WBAN = 722175, 13860
k_nearest = 5

5

In [3]:
using Revise
import TempModel

┌ Info: Recompiling stale cache file /Users/imolk/Library/Julia/compiled/v0.7/TempModel/5V5SX.ji for TempModel [429a4ede-8e99-57b3-891e-c3971593d88c]
└ @ Base loading.jl:1185


In [4]:
data_dir = "../data"
epsg = 3857 # Web Mercator (m)
isdList = TempModel.read_isdList(; data_dir=data_dir, epsg=epsg)
isd_wData = TempModel.stations_with_data(isdList; data_dir=data_dir)
isd_nearest_and_test = TempModel.find_nearest(isd_wData, USAF, WBAN, k_nearest)
isd_nearest = isd_nearest_and_test[2:end,:]

hourly_data = TempModel.read_Stations(isd_nearest; data_dir=data_dir)
;

In [5]:
k_spatiotemporal, logNoise = k_spatiotemporal = TempModel.fitted_sptemp_matern()
;

In [6]:
@time opt_out = TempModel.optim_kernel(k_spatiotemporal, logNoise, isd_nearest, hourly_data, :Optim);

20880.

  likely near /Users/imolk/Library/Julia/packages/IJulia/8Y05k/src/kernel.jl:41
  likely near /Users/imolk/Library/Julia/packages/IJulia/8Y05k/src/kernel.jl:41


152039 seconds (28.81 M allocations: 17.438 GiB, 0.08% gc time)


In [8]:
opt_out

Dict{Symbol,Any} with 4 entries:
  :mll      => -72132.8
  :hyp      => [-1.55879, -0.13913, 1.29157, 13.5471, -1.52905, -0.767335, -0.3…
  :logNoise => -1.55879
  :opt_out  => Results of Optimization Algorithm…

In [9]:
import JSON
save_dir = "."
GPmodel = "matern"
output_dictionary = Dict{String,Any}(
    "mll" => opt_out[:mll],
    "hyp" => opt_out[:hyp],
    "logNoise" => opt_out[:logNoise],
    "test_ICAO" => ICAO,
    "test_USAF" => USAF,
    "test_WBAN" => WBAN,
    "nearby_ICAO" => isd_nearest[:ICAO],
    "nearby_USAF" => isd_nearest[:USAF],
    "nearby_WBAN" => isd_nearest[:WBAN],
    "GPmodel" => GPmodel
)

savemodel_dir = joinpath(save_dir, "fitted_kernel", GPmodel)
if !isdir(savemodel_dir)
    mkdir(savemodel_dir)
end
fname = @sprintf("hyperparams_%s_%s.json", GPmodel, ICAO) 
filepath = joinpath(savemodel_dir, fname)
open(filepath, "w") do io
    indent = 4
    JSON.print(io, output_dictionary, indent)
end

Add `using Printf` to your imports.
  likely near /Users/imolk/Library/Julia/packages/IJulia/8Y05k/src/kernel.jl:41
